# Calcul de valeurs propres de Maxwell dans un carré
$\mathrm{curl\, curl}\, u = \lambda u$ dans le carré $(0,\pi) \times\ (0,\pi)$
avec condition limite magnétique homogène $u\times n =0$
Cf Arnold FEEC chap 1


In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from mpi4py import MPI

In [3]:
import dolfinx
print(f"DOLFINx version: {dolfinx.__version__} based on GIT commit: {dolfinx.git_commit_hash} of https://github.com/FEniCS/dolfinx/")

DOLFINx version: 0.7.3 based on GIT commit: ubuntu of https://github.com/FEniCS/dolfinx/


In [4]:
from dolfinx import mesh
from ufl import TrialFunction, TestFunction, grad, dx, dot
from slepc4py import SLEPc
from petsc4py.PETSc import ScalarType
from dolfinx.fem import (Constant, dirichletbc, Function, FunctionSpace, locate_dofs_geometrical,
                         locate_dofs_topological) 
from dolfinx.mesh import CellType,DiagonalType, create_rectangle, create_unit_square, locate_entities_boundary
from ufl import (FacetNormal, FiniteElement, Identity, Measure, TestFunction, TrialFunction, VectorElement,
                 as_vector, div, dot, ds, dx, inner)


In [5]:
#maillage uniforme diagonal
#domain = mesh.create_rectangle(MPI.COMM_WORLD,[np.array([0.0, 0.0]), np.array([np.pi, np.pi])], [40, 40], cell_type=CellType.triangle,
#                               diagonal=DiagonalType.right) 
#maillage uniforme crisscross
domain = mesh.create_rectangle(MPI.COMM_WORLD,[np.array([0.0, 0.0]), np.array([np.pi, np.pi])], [40, 40], cell_type=CellType.triangle,
                               diagonal=DiagonalType.crossed)                

In [6]:
domain.ufl_cell()

triangle

In [7]:
tdim = domain.topology.dim
fdim = tdim - 1
domain.topology.create_connectivity(fdim, tdim)
facets = mesh.exterior_facet_indices(domain.topology)


In [8]:
element = VectorElement("CG", domain.ufl_cell(), 1, tdim)
V = FunctionSpace(domain, element)

import ufl
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)
#u = TrialFunction(V)
#v = TestFunction(V)

In [9]:
def Rot(v):
    return v[1].dx(0) - v[0].dx(1)

In [10]:
a= ufl.dot(ufl.curl(u), ufl.curl(v))*ufl.dx
#a= Rot(u)*Rot(v)*ufl.dx
b = ufl.dot(u,v)*ufl.dx
from dolfinx import fem
bilinear_form = fem.form(a)
mass_matrix = fem.form(b)
#a = dot(grad(u), grad(v))*dx
# form = Form(a)

In [11]:
def hori(x):
    return np.logical_or( np.isclose(x[1], 0), np.isclose(x[1],np.pi))
def vert(x):
    return np.logical_or( np.isclose(x[0], 0), np.isclose(x[0],np.pi))

hori_facets = locate_entities_boundary(domain, fdim, hori)
vert_facets = locate_entities_boundary(domain, fdim, vert)
boundary_dofs_x = locate_dofs_topological(V.sub(0), fdim, hori_facets)
boundary_dofs_y = locate_dofs_topological(V.sub(1), fdim, vert_facets)
bcx = dirichletbc(ScalarType(0), boundary_dofs_x, V.sub(0))                     
bcy = dirichletbc(ScalarType(0), boundary_dofs_y, V.sub(1)) 
# uxn =0 composante tangentielle nulle
bct = [bcx, bcy]



In [12]:
import dolfinx.fem.petsc
# Assemble stiffness tensor and mass matrix
A = fem.petsc.assemble_matrix(bilinear_form, bct)
# attention à ne pas mettre de conditions limites dans
#la matrice de masse sinon on a 1 valeur propre parasite 
# une autre façon plus propre est de
#  mettre la diagonale à 0 sur les noeuds du bord
B = fem.petsc.assemble_matrix(mass_matrix, bct, diagonal=0.0)
#B = fem.petsc.assemble_matrix(mass_matrix)
A.assemble()
B.assemble()


In [13]:
eigensolver = SLEPc.EPS().create(MPI.COMM_WORLD)
#eigensolver = SLEPc.EPS().create(domain.comm)


In [14]:
eigensolver.setOperators(A,B)
#GHEP means generalized hermitian problem
eigensolver.setProblemType(SLEPc.EPS.ProblemType.GHEP)
#eigensolver.setType(SLEPc.EPS.Type.LAPACK)
eigensolver.setType(SLEPc.EPS.Type.KRYLOVSCHUR)
#nombre de valeurs propres
nev=25
eigensolver.setDimensions(nev)
#les plus petites valeurs propres en module
#eigensolver.setWhichEigenpairs(2)
#les plus grandes valeurs propres en module
#eigensolver.setWhichEigenpairs(1)
eigensolver.setWhichEigenpairs(eigensolver.Which.TARGET_MAGNITUDE)
shift = 5.5
# eigenvalue close to 5.5
eigensolver.setTarget( shift ) 
#spectral shift
st = eigensolver.getST()
st.setType(SLEPc.ST.Type.SINVERT)
st.setShift(shift)

In [15]:
eigensolver.solve()

In [16]:
evs = eigensolver.getConverged()
print( "Number of converged eigenpairs %d" % evs )


Number of converged eigenpairs 33


In [17]:
n_conv = eigensolver.getConverged()
computed_eigenvalues = []
for i in range(min(n_conv, nev)):
    lmbda = eigensolver.getEigenvalue(i)
    computed_eigenvalues.append(np.round(np.real(lmbda), 2))
np.sort(computed_eigenvalues)

array([ 0.  ,  0.  ,  0.  , -0.  , -0.  ,  0.  ,  1.  ,  1.  ,  2.  ,
        4.  ,  4.  ,  5.  ,  5.  ,  6.  ,  8.01,  9.01,  9.01, 10.02,
       10.02, 13.03, 13.03, 14.95, 14.95, 16.04, 16.04])

### On retrouve  exactement les résultats de Arnold p9 
en particulier la valeur propre 6 est fausse ainsi que la valeur propre double 15

In [18]:
import pyvista
print(pyvista.global_theme.jupyter_backend)


trame


In [19]:
#pyvista.set_jupyter_backend('trame')
from dolfinx import plot
tdim = domain.topology.dim
pyvista.start_xvfb()
topology, cell_types, geometry = plot.vtk_mesh(domain, tdim)
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)


In [20]:
plotter = pyvista.Plotter()
plotter.add_mesh(grid, show_edges=True)
plotter.view_xy()
if not pyvista.OFF_SCREEN:
    plotter.show()
else:
    figure = plotter.screenshot("fundamentals_mesh.png")


Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


### les valeurs propres exactes
$\nabla \times \nabla  u = \lambda u$ sur $(0,\pi) \times (0,\pi)$ avec condition limite magnétique $u \times n =0$
sont données par $u(x,y) =  \mathrm{curl}\sin(k x) \sin(l y)$ 
et 
$\lambda = (k^2 + l^2)$ pour $k,l \geq 0$ excepté $k=l=0$

In [21]:
list = []
for k in range(0,5):
    for l in range(0,5):
        lamb = (k**2 + l**2)
        list.append(lamb)
list.sort()
print(list[1:])

[1, 1, 2, 4, 4, 5, 5, 8, 9, 9, 10, 10, 13, 13, 16, 16, 17, 17, 18, 20, 20, 25, 25, 32]
